In [ ]:
!pip install pandas pyspark neo4j matplotlib yfiles_jupyter_graphs_for_neo4j neo4j-graphrag

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.3/111.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 51.3 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=36dee4d92f906681f106901c2b93a1690685e9826b3c362dc5c74b1176ef495c
  Stored in directory: /root/.cache/p

In [ ]:
!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 11.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, count
from functools import reduce
from neo4j import GraphDatabase
import pandas as pd
import numpy as np

In [ ]:
# Initialize Spark session
spark = SparkSession.builder.appName("Big Data Project 2").getOrCreate()

In [ ]:
uri = "neo4j+s://f7da033b.databases.neo4j.io:7687"
username = "neo4j"
password = "qeQGXKiNL5AdRQY_y_mGcyeKTE3FxfdKmsiOTXneS9U"

driver = GraphDatabase.driver(uri, auth=(username, password))

def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return result.data()

# Test connection by running a sample query
test_query = "RETURN 'Connection Successful' AS message"
print(run_query(test_query))

[{'message': 'Connection Successful'}]


# **Q4**

In [ ]:
sub_df_q1 = spark.read.csv('/content/drive/MyDrive/Fall 24 (sem 3)/Big data/Project 2/dataset/2017q1/sub.txt', sep='\t', header=True, inferSchema=True)
sub_df_q2 = spark.read.csv('/content/drive/MyDrive/Fall 24 (sem 3)/Big data/Project 2/dataset/2017q2/sub.txt', sep='\t', header=True, inferSchema=True)
sub_df_q3 = spark.read.csv('/content/drive/MyDrive/Fall 24 (sem 3)/Big data/Project 2/dataset/2017q3/sub.txt', sep='\t', header=True, inferSchema=True)
sub_df_q4 = spark.read.csv('/content/drive/MyDrive/Fall 24 (sem 3)/Big data/Project 2/dataset/2017q4/sub.txt', sep='\t', header=True, inferSchema=True)

sub_df = sub_df_q1.union(sub_df_q2).union(sub_df_q3).union(sub_df_q4)

filter_form_df = sub_df.where(col('form').isin(['8-K','10-K']))

In [ ]:
latest_cik_df = filter_form_df.groupBy('cik') \
    .agg(F.max('accepted').alias('latest_accepted'))

filtered_df = filter_form_df.join(latest_cik_df,
                              (filter_form_df.cik == latest_cik_df.cik) &
                              (filter_form_df.accepted == latest_cik_df.latest_accepted),
                              'inner') \
    .select(filter_form_df['*'])

filtered_df.count()

6003

In [ ]:
#@title Get urls

import requests
from bs4 import BeautifulSoup
import time

headers = {
    'User-Agent': 'MyCustomUserAgent/1.0 (madankumar@gwu.edu)'
}

url_list=[]
dir_url_list = [
    [row.name, f"https://www.sec.gov/Archives/edgar/data/{row.cik}/{row.adsh.replace('-', '')}"]
    for row in filtered_df.collect()  # Collect rows from DataFrame
]

for comp_name, url in dir_url_list:

  # Fetch the page content
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.content, 'html.parser')

  # Initialize variables to track the largest file
  largest_file = None
  largest_size = 0

  # Find all file links in the directory
  for row in soup.find_all('tr'):
      cells = row.find_all('td')

      if len(cells) >= 2:  # Ensure there are at least two cells (link and size)
          link = cells[0].find('a')

          if link and link['href'].endswith('.htm') :
              # Get the size of the file
              size_str = cells[1].text.strip()  # Get the size from the second cell
              size = int(size_str.replace(',', ''))  # Convert size to an integer

              # Check if this file is the largest
              if size > largest_size:
                  largest_size = size
                  largest_file = link['href']

  if largest_file:
      url_list.append([comp_name, 'https://www.sec.gov'+largest_file])


url_list

[['HURCO COMPANIES INC',
  'https://www.sec.gov/Archives/edgar/data/315374/000114420417001161/v455086_10k.htm'],
 ['TELEHEALTHCARE, INC.',
  'https://www.sec.gov/Archives/edgar/data/1628104/000147793217000060/tllt_10k.htm'],
 ['EXOLIFESTYLE, INC.',
  'https://www.sec.gov/Archives/edgar/data/1343465/000149315217000281/form10-k.htm'],
 ['PASSUR AEROSPACE, INC.',
  'https://www.sec.gov/Archives/edgar/data/225628/000109690617000013/passur.htm'],
 ['LIMONEIRA CO',
  'https://www.sec.gov/Archives/edgar/data/1342423/000114420417001591/v455916_10k.htm'],
 ['CROWDGATHER, INC.',
  'https://www.sec.gov/Archives/edgar/data/1328670/000157587217000007/10k.htm'],
 ['VOIP-PAL.COM INC',
  'https://www.sec.gov/Archives/edgar/data/1410738/000158069517000021/vplm-10k_093016.htm'],
 ['VOLT INFORMATION SCIENCES, INC.',
  'https://www.sec.gov/Archives/edgar/data/103872/000010387217000006/visi-10302016x10k.htm'],
 ['FUELCELL ENERGY INC',
  'https://www.sec.gov/Archives/edgar/data/886128/000088612817000007/fce

In [ ]:
len(url_list)

6003

In [ ]:
#@title save url_lists

filename = '/content/drive/MyDrive/Fall 24 (sem 3)/Big data/Project 2/dataset/temp.txt'

# Open the file for writing
with open(filename, 'w') as f:
    for entry in url_list:
        f.write(f"{entry[0]}|| {entry[1]}\n")

print(f"Data saved to {filename}")

Data saved to /content/drive/MyDrive/Fall 24 (sem 3)/Big data/Project 2/dataset/url_list.txt


In [ ]:
#@title retrieve urls
filename = '/content/drive/MyDrive/Fall 24 (sem 3)/Big data/Project 2/dataset/url_list.txt'

url_list = []

# Open the file for reading
with open(filename, 'r') as f:
    for line in f:
        name, url = line.strip().split('|| ', 1)
        url_list.append([name, url])

len(url_list)

6003

In [ ]:
#@title Segment office holders

import requests
from bs4 import BeautifulSoup

headers = {
    'User-Agent': 'MyCustomUserAgent/1.0 (madankumar@gwu.edu)'
}

def fetch_office_holder_data(url):
    # Fetch the .htm file
    response = requests.get(url,headers=headers)

    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        office_holder_sections = []

        target_text = "this report has been signed below by the following persons on behalf of the Registrant"

        table = None
        for tag in soup.find_all(True):  # True means all tags
            tag_text = ' '.join(tag.get_text().split()).lower()
            if target_text.lower() in tag_text:
                # Found the tag, now get the next table with office holders
                table = tag.find_next('table')

        if table:

            s_tags = table.find_all(string=lambda text: text and text.startswith('/s/'))

            for i, s_tag in enumerate(s_tags):
                # Get the parent tag of the /s/ text
                start_tag = s_tag.find_parent()

                # Prepare to collect tags until the next /s/ occurrence
                section = [str(start_tag)]

                # Look for the next /s/ tag
                next_s_tag = s_tags[i + 1] if i + 1 < len(s_tags) else None

                # If a next /s/ tag exists, collect until that tag
                if next_s_tag:
                    current_tag = start_tag.find_next()

                    while current_tag and current_tag != next_s_tag.find_parent():
                        if(str(current_tag).find('/s/'))!=-1:
                            break
                        section.append(str(current_tag))
                        current_tag = current_tag.find_next()

                else:
                    # Collect till the end of the document
                    current_tag = start_tag.find_next()
                    while current_tag:
                        section.append(str(current_tag))
                        current_tag = current_tag.find_next()

                # Join the collected section to create HTML
                office_holder_sections.append(''.join(section))

        return office_holder_sections
    else:
        return []


In [ ]:
#@title EXTRACT ALL DATA FROM ALL TAGS

def extract_text_from_html(office_holder_data):
    all_texts = []

    for html_str in office_holder_data:
        soup = BeautifulSoup(html_str, 'html.parser')

        texts = []
        for element in soup.find_all(True):  # All tags
            # Get text, strip whitespace, and handle it correctly
            text = element.get_text(separator='|', strip=True)  # Use '|' as a separator
            if text:  # Only add non-empty text
                texts.extend(text.split('|'))  # Split by the separator

        # Filter and clean up the texts
        texts = [t for t in texts if t]  # Remove empty entries

        # Append non-empty texts to the list
        all_texts.append(texts)

    return all_texts


In [ ]:
#@title DICT SET OF NAME AND TITLES

import re
import concurrent.futures


def create_executives(executive_data):
  with driver.session() as session:
    for company, executives in executive_data.items():
      for name, titles in executives.items():
        for title in titles:
          title_str = title.replace(" ", "_").upper()
          query = f"""
              MERGE (c:Company {{name: $company}})
              MERGE (e:Executive {{name: $name}})
              MERGE (e)-[:{title_str}]->(c)
          """
          session.run(query, name=name, company=company)


titles_dict = {
    'VP': ['Vice President', 'VP'],
    'PRESIDENT': ['President'],
    'MANAGER': ['Manager'],
    'DIRECTOR': ['Director'],
    'EXECUTIVE CHAIRMAN OF THE BOARD': ['Executive Chairman of the Board', 'Chairman of the Board'],
    'CHAIRMAN': ['Chairman'],
    'SECRETARY': ['Secretary'],
    'TREASURER': ['Treasurer'],
    'CEO': ['Chief Executive Officer', 'Chief Executive', 'CEO', 'Executive Officer'],
    'CAO': ['Chief Accounting Officer', 'Chief Accounting'],
    'PAO': ['Principal Accounting Officer', 'Principal Accounting', 'Accounting Officer'],
    'PFO': ['Principal Financial Officer', 'Principal Financial', 'Financial Officer'],
    'CFO': ['Chief Financial Officer', 'Chief Financial'],
    'PEO': ['Principal Executive Officer', 'Principal Executive', 'Executive Officer'],
    'CTO': ['Chief Technology Officer', 'Chief Technology', 'Technology Officer'],
    'CIO': ['Chief Information Officer', 'Chief Information', 'Information Officer'],
    'COO': ['Chief Operating Officer', 'Chief Operating' , 'Operating Officer'],
}

def extract_names_and_titles(names_titles_list):
    name_title_dict = {}

    # Define the regex pattern for splitting titles
    pattern = r'[,&/()]+| and '

    for name_titles in names_titles_list:

        name = None
        title_set = set()

        # Iterate through the list
        for item in name_titles:
            # Check if the item is a name (starts with /s/)
            if item[:3] == ('/s/'):
                # Clean the name
                name = item[3:].strip()  # Remove "/s/" and strip spaces

                if name.find('&') != -1 or name.find(',') != -1 or len(name)<5:
                    name = None
                    continue

            else:
                # Add the title to the current name's set
                if name:
                    title = item.strip().lower().replace(name.lower(),'').strip()
                    if title and not any(char.isdigit() for char in title):  # Avoid adding titles with digits
                        titles_hold = [' '.join(t.strip().split()) for t in re.split(pattern, title) if t.strip()]

                        for key, val in titles_dict.items():
                            for t in titles_hold:
                                if any (v.lower() in t.lower() for v in val):
                                    title_set.add(key)

        if all(t in title_set for t in ('PRESIDENT', 'VP')):
            title_set.remove('PRESIDENT')

        if all(t in title_set for t in ('CHAIRMAN', 'EXECUTIVE CHAIRMAN OF THE BOARD')):
            title_set.remove('CHAIRMAN')


        if name and title_set:
            name_title_dict[name] = title_set

    return name_title_dict

comp_i = 1

def process_company(comp_name, url):
    office_holder_data = fetch_office_holder_data(url)
    names_titles_list = extract_text_from_html(office_holder_data)
    name_title_dict = extract_names_and_titles(names_titles_list)

    if name_title_dict:
        create_executives({comp_name: name_title_dict})  # Push to Neo4j

    return comp_name, name_title_dict


comp_office_holders = {}


with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_company, comp_name, url): comp_name for comp_name, url in url_list}

    for i, future in enumerate(concurrent.futures.as_completed(futures), start=1):
        comp_name, name_title_dict = future.result()
        if name_title_dict:
            comp_office_holders[comp_name] = name_title_dict
            print(f'got {comp_i} comp executives')
            comp_i += 1


got 1 comp executives
got 2 comp executives
got 3 comp executives
got 4 comp executives
got 5 comp executives
got 6 comp executives
got 7 comp executives
got 8 comp executives
got 9 comp executives
got 10 comp executives
got 11 comp executives
got 12 comp executives
got 13 comp executives
got 14 comp executives
got 15 comp executives
got 16 comp executives
got 17 comp executives
got 18 comp executives
got 19 comp executives
got 20 comp executives
got 21 comp executives
got 22 comp executives
got 23 comp executives
got 24 comp executives
got 25 comp executives
got 26 comp executives
got 27 comp executives
got 28 comp executives
got 29 comp executives
got 30 comp executives
got 31 comp executives
got 32 comp executives
got 33 comp executives
got 34 comp executives
got 35 comp executives
got 36 comp executives
got 37 comp executives
got 38 comp executives
got 39 comp executives
got 40 comp executives
got 41 comp executives
got 42 comp executives
got 43 comp executives
got 44 comp executiv

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

# **Displaying Graph**

In [ ]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget

g = Neo4jGraphWidget(driver)

def show_graph(driver):
    query = """
    MATCH (e:Executive)-[r]->(c:Company)
    WHERE type(r) IN [
        'CAO',
        'CEO',
        'CFO',
        'CHAIRMAN',
        'CIO',
        'COO',
        'CTO',
        'DIRECTOR',
        'EXECUTIVE_CHAIRMAN_OF_THE_BOARD',
        'MANAGER',
        'PAO',
        'PEO',
        'PFO',
        'PRESIDENT',
        'SECRETARY',
        'TREASURER',
        'VP'
    ]
    AND NOT e.name CONTAINS 'LLP'
    WITH e, COUNT(DISTINCT c) AS companyCount
    WHERE companyCount > 1
    MATCH (e)-[r]->(c:Company)
    RETURN e, r, c
    """
    g.show_cypher(query)

show_graph(driver)

GraphWidget(layout=Layout(height='800px', width='100%'))

**Process Overview:**

- Combine the quarterly DataFrames and filter for specific SEC forms ('8-K', '10-K').
- Identify the latest accepted filings for each CIK, isolating the most recent submissions.
- Create a list of URLs to access the SEC filings, focusing on the largest .htm files in each directory.
- Fetch and parse the filing documents to extract office holders names and titles.
- Use a mapping of titles to categorize and unify various executive titles.
- Extract and categorize executives.
- Store relationships between executives and companies in the Neo4j database.
- Execute a query to count how many companies each executive is associated with.


In [ ]:
#@title Centrality

q4_query = """
MATCH (e:Executive)-[r]->(c:Company)
WHERE type(r) IN [
    'CAO',
    'CEO',
    'CFO',
    'CHAIRMAN',
    'CIO',
    'COO',
    'CTO',
    'DIRECTOR',
    'EXECUTIVE_CHAIRMAN_OF_THE_BOARD',
    'MANAGER',
    'PAO',
    'PEO',
    'PFO',
    'PRESIDENT',
    'SECRETARY',
    'TREASURER',
    'VP'
]
AND NOT e.name CONTAINS 'LLP'
WITH e, COUNT(c) AS companyCount
WHERE companyCount > 1
RETURN e.name AS Executive, companyCount
ORDER BY companyCount DESC
"""


data_q4 = run_query(q4_query)

df_q4 = spark.createDataFrame(data_q4)

df_q4.show(10)

+--------------------+------------+
|           Executive|companyCount|
+--------------------+------------+
|   JEFFREY S. SAVAGE|          23|
|   ROBERT E. SWANSON|          20|
|       Denis McGlynn|          18|
|   Gregory B. Maffei|          16|
|    Benjamin M. Fink|          14|
|           Kimun Lee|          14|
|  SCOTT M. PROCHAZKA|          13|
|   ALLEN L. LEVERETT|          12|
|  Richard A. Hubbell|          12|
|JEFFREY K. WALDVOGEL|          12|
+--------------------+------------+
only showing top 10 rows



Executives Jeffrey S. Savage and Robert E. Swanson have significant influence, holding positions across many companies, indicating their industry prominence and potential strategic impact.

In [ ]:
df_q4.count()

3818

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [ ]:
#@title CLEAR DB


class Neo4jDatabase:
    def __init__(self, uri, username, password):
        self.driver = GraphDatabase.driver(uri, auth=(username, password))

    def reset_database(self):
        with self.driver.session() as session:
            # Run a Cypher query to delete all nodes and relationships
            session.run("MATCH (n) DETACH DELETE n")
            print("Database has been reset.")

    def close(self):
        self.driver.close()


# Initialize the database connection
neo4j_db = Neo4jDatabase(uri, username, password)

# Reset the database
neo4j_db.reset_database()

# Close the connection
neo4j_db.close()

Database has been reset.
